# Reductions on GPUs

A common computational pattern in many applications is the *reduction*.
Examples are summing all elements of a vector, computing a dot product, or finding a minimum value in an array.

## The Problem

On GPUs, the main challenge in parallelizing reductions is the inherent race condition.
Consider the following CPU function and its corresponding GPU kernel:

```cpp
void reduce(double *data, size_t nx) {
    double sum = 0;

    for (size_t i0 = 0; i0 < nx; ++i0)
        sum += data[i0];

    return sum;
}
```

```cpp
__global__ void reduce(double *data, double* sum, size_t nx) {
    const size_t i0 = blockIdx.x * blockDim.x + threadIdx.x;

    if (i0 < nx)
        *sum += data[i0];
}
```

The compound assignment appears like a single operation, but actually involves several steps:
* Loading the old value of `sum` from memory into a temporary variable (e.g., `tmp`)
* Adding `data[i0]` to `tmp`
* Writing the value of `tmp` back to `sum`

When these steps are performed in parallel, some updates may be lost:
* Multiple threads read `sum` concurrently
* Each modifies its local copy
* They write back their results, potentially overwriting each other's contributions

## The Solution in CUDA

One solution is to ensure the compound assignment is performed as a single *atomic* operation:

```cpp
__global__ void reduce(double *data, double* sum, size_t nx) {
    const size_t i0 = blockIdx.x * blockDim.x + threadIdx.x;

    if (i0 < nx)
        atomicAdd(sum, data[i0]);
}
```

While this version is correct, performance may suffer due to *atomic congestion*.
Further optimization is possible through *hierarchical reduction*, which can include:
* Each thread summing multiple input values
* Each warp performing a reduction across its threads
* Each block performing a reduction across its threads or warps

A full discussion of all variants is beyond this tutorial, but a practical option is to use a block reduction with `cub`, a header-only library included in the Nvidia HPC Toolkit (or `hipCUB` on AMD):

```cpp
#include <cub/cub.cuh>

template <unsigned int blockSize>
__global__ void reduce(double *data, double* sum, size_t nx) {
    const size_t i0 = blockIdx.x * blockDim.x + threadIdx.x;

    // Define BlockReduce type for the block size
    typedef cub::BlockReduce <double, blockSize, cub::BLOCK_REDUCE_RAKING_COMMUTATIVE_ONLY> BlockReduce;

    // Allocate shared memory for block reduction
    __shared__ typename BlockReduce::TempStorage tempStorage;

    double elem = 0;

    if (i0 < nx)
        elem = data[i0];

    // Reduce within the block (all threads *must* participate)
    double blockSum = BlockReduce(tempStorage).Sum(elem);

    // Atomically add the result to the global sum
    if (0 == threadIdx.x && i0 < nx)
        atomicAdd(sum, blockSum);
}
```

Other programming models also support reductions, with varying levels of programming effort, flexibility, and performance.

## OpenMP

```cpp
double sum = 0;

#pragma omp target teams distribute parallel for \
            reduction(+ : sum)
for (size_t i0 = 0; i0 < nx; ++i0)
    sum += data[i0];
```

## OpenACC

```cpp
double sum = 0;

#pragma acc parallel loop present(data[:nx]) \
            reduction(+ : sum)
for (size_t i0 = 0; i0 < nx; ++i0)
    sum += data[i0];
```

## Modern C++

```cpp
double sum = std::reduce(std::execution::par_unseq, data, data + nx, 0., std::plus<>{});
```

## Thrust

```cpp
double sum = thrust::reduce(data, data + nx, 0.);
```

## Kokkos

```cpp
double sum = 0;

Kokkos::parallel_reduce(
    Kokkos::RangePolicy<>(0, nx),
    KOKKOS_LAMBDA(const size_t i0, double &acc) {
        acc += data(i0);
    }, sum);
```

## SYCL

```cpp
q.submit([&](sycl::handler &h) {
    h.parallel_for(nx, [=](auto i0) {
        auto v = sycl::atomic_ref<double, sycl::memory_order::relaxed,
                                  sycl::memory_scope::device,
                                  sycl::access::address_space::global_space>(
            sum[0]);
        v.fetch_add(data[i0]);
    });
});
```

Additional optimizations, similar to CUDA, are possible.
For more information, see [Intel's OneAPI optimization guide](https://www.intel.com/content/www/us/en/docs/oneapi/optimization-guide-gpu/2025-0/reduction.html).

## Additional Consideration

Often, reductions can be fused with the production of their input values.
For example, when computing a dot product, a naive implementation performs two steps:
* Compute the point-wise multiplication and store the result in a temporary vector
* Apply a sum reduction to the temporary vector

Fusing these steps reduces memory usage and typically improves performance by minimizing data movement.

Most of the approaches discussed above support this fusion easily.
Two exceptions are modern C++ and Thrust, which require different algorithms:
* `std::transform_reduce`
* `thrust::transform_reduce` or `thrust::transform_iterator`